In [12]:
import torch
import pygame
import numpy as np
import os

# Directory for saving frames

frame_directory = "frames"
os.makedirs(frame_directory, exist_ok=True)

# Ensure that CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Simulation parameters
width, height = 400, 400
Nx, Ny = 100, 100
dx, dy = 1.0 / Nx, 1.0 / Ny
dt = 0.01  # Time step
viscosity = 0.02

x_center_1, y_center_1 = 0.5, 0.5  # Center of the 1st Gaussian
x_center_2, y_center_2 = 1.5, 0.5  # Center of the 2nd Gaussian

sigma_1 = 0.10  # Standard deviation of the 1st Gaussian
sigma_2 = 0.20  # Standard deviation of the 2nd Gaussian

magnitude = 200  # Magnitude of the temperature increase

# Initialize the temperature array with two Gaussian distributions

X, Y = torch.meshgrid(torch.linspace(0, 2, Nx), torch.linspace(0, 1, Ny))


In [13]:

gaussian_field_1 = magnitude * torch.exp(-((X - x_center_1)**2 + (Y - y_center_1)**2) / (2 * sigma_1**2))
gaussian_field_2 = magnitude * torch.exp(-((X - x_center_2)**2 + (Y - y_center_2)**2) / (2 * sigma_2**2))
# plt.matshow(gaussian_field_1
#             + gaussian_field_2)
# plt.show()
T = (200 + gaussian_field_1 + gaussian_field_2).unsqueeze(0).unsqueeze(1).to(device)


In [14]:
T.shape


torch.Size([1, 1, 100, 100])

In [15]:
T_pad = torch.functional.F.pad(T, (0, 0, 0, 0), mode="circular")

In [16]:
T_pad.shape

torch.Size([1, 1, 100, 100])

In [17]:
def kernel(a_x, a_y, dt):
    return dt*torch.tensor([
                        [0, a_y, 0],
                        [a_x, -2*(a_x + a_y), a_x],
                        [0, a_y, 0]
                        ], dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(1)

In [18]:
a_x = viscosity / dx**2
a_y = viscosity / dy**2
k = kernel(a_x, a_y, dt)

In [19]:
laplacian = torch.nn.functional.conv2d(T_pad, k, stride=1, padding = 1)
laplacian.shape

torch.Size([1, 1, 100, 100])

In [20]:
k

tensor([[[[ 0.,  2.,  0.],
          [ 2., -8.,  2.],
          [ 0.,  2.,  0.]]]], device='cuda:0')

In [21]:
class HeatEquationSolver(torch.nn.Module):
    def __init__(self, T, kernel, dt, dx, dy, viscosity):
        self.T = T
        self.kernel = kernel
        self.dt = dt
        self.dx = dx
        self.dy = dy
        self.viscosity = viscosity

    def forward(self, T):
        # Pad the temperature array
        T_pad = torch.clone(self.T)
        T = torch.clone(self.T)
        T_pad = torch.functional.F.pad(T_pad, (0, 0, 0, 0), mode="circular")
        # Compute the Laplacian
        laplacian = torch.nn.functional.conv2d(T_pad, self.kernel, stride=1, padding = 1)

        
        

        # Compute the new temperature
        T += self.dt * laplacian
        self.T = torch.clone(T)
        return T       


In [25]:
# Pygame setup
pygame.init()
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

running = True
frame_number = 0
update_skip = 2
MULT = 4
solver = HeatEquationSolver(T, k, dt, dx, dy, viscosity)
last_T = solver.T
while running:

    with torch.no_grad():
        for i in range(MULT):
            last_T = solver.forward(last_T)
        # T = solver.forward()
        # solver.step()
        # print(torch.allclose(solver.T, last_T))
        print(torch.max(solver.T), torch.min(solver.T))
        # Transfer data to CPU for visualization and convert to NumPy
        # T_cpu = solver.T.repeat(1, 3, 1, 1)

        T_cpu = last_T.cpu().detach().squeeze(1).squeeze(0).numpy()
        print(T_cpu.shape, T_cpu.min())
        T_color = T_cpu - np.min(T_cpu)
        T_color = T_color / np.max(T_color)
        T_color = np.uint8(255 * T_color)
        # print(T_color.shape, np.max(T_color))
        from matplotlib import pyplot as plt
        plt.imshow(T_color)
        plt.savefig(f"frame_{frame_number:04d}.png")

        # T_cpu = solver.T.cpu().squeeze(1).numpy()
        # Visualization
        for i in range(Nx):
            for j in range(Ny):
                color = T_color[i, j]
                # print(T_color[i, j])
                # color = min(0, max(0, int(255 * color)))
                # pygame.draw.rect(screen, (color, color, color), (i * width / Nx, j * height / Ny, width / Nx, height / Ny))

        # pygame.surfarray.blit_array(screen, T_cpu)
        # pygame.display.flip()
        clock.tick(60)
        
        # Save frame
        # pygame.image.save(screen, os.path.join(frame_directory, f"frame_{frame_number:04d}.png"))
        frame_number += 1
        # print(frame_number)
        # Check for Pygame quit events
        # for event in pygame.event.get():
        #     if event.type == pygame.QUIT:
        #         running = False

    pygame.quit()


tensor(399.6693, device='cuda:0') tensor(170.7421, device='cuda:0')
(100, 100) 170.74214
tensor(399.4666, device='cuda:0') tensor(147.6604, device='cuda:0')
(100, 100) 147.66042
tensor(399.2644, device='cuda:0') tensor(129.2012, device='cuda:0')
(100, 100) 129.2012
tensor(399.0627, device='cuda:0') tensor(114.2479, device='cuda:0')
(100, 100) 114.24794
tensor(398.8615, device='cuda:0') tensor(101.9878, device='cuda:0')
(100, 100) 101.98782
tensor(398.6607, device='cuda:0') tensor(91.8217, device='cuda:0')
(100, 100) 91.82166
tensor(398.4604, device='cuda:0') tensor(83.3024, device='cuda:0')
(100, 100) 83.30244
tensor(398.2606, device='cuda:0') tensor(76.0928, device='cuda:0')
(100, 100) 76.0928
tensor(398.0612, device='cuda:0') tensor(69.9353, device='cuda:0')
(100, 100) 69.93534
tensor(397.8622, device='cuda:0') tensor(64.6316, device='cuda:0')
(100, 100) 64.6316
tensor(397.6638, device='cuda:0') tensor(60.0270, device='cuda:0')
(100, 100) 60.02698
tensor(397.4659, device='cuda:0') te